<a href="https://colab.research.google.com/github/cantabile1129/VCII_CG_finalreport/blob/main/colab/websocket_server.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

!apt-get install libnvidia-gl-$(grep -oP 'NVIDIA UNIX x86_64 Kernel Module\s+\K[\d.]+(?=\s+)' /proc/driver/nvidia/version | grep -oE '^[0-9]+')


grep: /proc/driver/nvidia/version: No such file or directory
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Virtual packages like 'libnvidia-gl' can't be removed
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [2]:
%%file requirements.txt
glcontext==2.5.0
moderngl==5.10.0
numpy
opencv-python-headless
websockets


Writing requirements.txt


In [3]:

!pip install -r requirements.txt
!pip install pyngrok ping3


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.7/51.7 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.3/269.3 kB 7.4 MB/s eta 0:00:00


In [4]:

!curl -sL https://raw.githubusercontent.com/MsrOhkwr/vc2-remote-rendering/main/app/__main__.py | python


moderngl 5.10.0
---------------
vendor: Mesa
renderer: llvmpipe (LLVM 15.0.7, 256 bits)
version: 4.5 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.3
python: 3.11.11 (main, Dec  4 2024, 08:55:07) [GCC 11.4.0]
platform: linux
code: 450


In [5]:

!mkdir -p app
!mkdir -p assets/glsl
!mkdir -p colab


In [6]:
%%file assets/glsl/vertex_shader.glsl
#version 330 core

in vec2 position_vertices;

void main() { gl_Position = vec4(position_vertices, 0.0f, 1.0f); }


Writing assets/glsl/vertex_shader.glsl


In [40]:
%%file assets/glsl/fragment_shader_path_trace.glsl
#version 330 core

#define POW2(X) ((X) * (X))
#define POW5(X) ((X) * (X) * (X) * (X) * (X))
#define DEPTH_MAX (16)
#define DELTA (0.01)
#define PI (3.14159265359)

#define BACKGROUND (0)
#define DIFFUSE (1)
#define MIRROR (2)
#define GLASS (3)

out vec4 input_color;
out uvec4 seed_value;

uniform sampler2D input_image;
uniform usampler2D seed_image;
uniform sampler2D background_image;

uniform int sample_max;
uniform int current_sample;
uniform float theta;
uniform float phi;
uniform float move_x;
uniform float move_y;

ivec2 group_num = ivec2($width, $height);
ivec2 group_idx = ivec2(gl_FragCoord.x, gl_FragCoord.y);

struct Ray {
  vec3 origin;    // 光線の始点
  vec3 direction; // 光線の方向ベクトル
  vec3 scatter;   // 散乱成分
  int depth;      // 反射した回数
};

struct Hit {
  float t;       // 光線の始点から衝突位置までの距離
  vec3 position; // 衝突位置
  vec3 normal;   // 衝突位置における法線ベクトル
  vec3 scatter;  // 散乱成分
  vec3 emission; // 放出成分
  int material;  // 材質
};

struct Sphere {
  vec3 center;   // 球の中心
  float radius;  // 球の半径
  vec3 scatter;  // 散乱成分
  vec3 emission; // 放出成分
  int material;  // 材質
};

uvec4 xors;

// Xorshift による疑似乱数生成
float rand() {
  uint t = (xors[0] ^ (xors[0] << 11));
  xors[0] = xors[1];
  xors[1] = xors[2];
  xors[2] = xors[3];
  xors[3] = (xors[3] ^ (xors[3] >> 19)) ^ (t ^ (t >> 8));
  return xors[3] / 4294967295.0f;
}

// 球と光線の交点
bool hitSphere(const in Sphere sphere, const in Ray ray, inout Hit hit) {
  // float a = dot(ray.direction, ray.direction);
  // ray.direction は単位ベクトルであり，必ず 1 になるので省略
  float b = dot(ray.origin, ray.direction) - dot(sphere.center, ray.direction);
  float c = dot(ray.origin, ray.origin) - 2 * dot(ray.origin, sphere.center) +
            dot(sphere.center, sphere.center) - POW2(sphere.radius);
  float d = POW2(b) - c;

  float t1, t2;
  t1 = abs(b) + sqrt(d);
  t1 = (b < 0) ? t1 : -t1;
  t2 = c / t1;

  float t;
  if (d > 0) {
    t = min(t1, t2);
    if (0 < t && t < hit.t) {
      hit.t = t;
      hit.position = ray.origin + t * ray.direction;
      hit.normal = normalize(hit.position - sphere.center);
      hit.scatter = sphere.scatter;
      hit.emission = sphere.emission;
      hit.material = sphere.material;
      return true;
    }
    t = max(t1, t2);
    if (0 < t && t < hit.t) {
      hit.t = t;
      hit.position = ray.origin + t * ray.direction;
      hit.normal = normalize(hit.position - sphere.center);
      hit.scatter = sphere.scatter;
      hit.emission = sphere.emission;
      hit.material = sphere.material;
      return true;
    }
  }

  return false;
}

// 鏡面
void mirror(inout Ray ray, const in Hit hit) {
  if (dot(-ray.direction, hit.normal) < 0) {
    ray.depth = DEPTH_MAX;
    return;
  }
  ray.depth++;
  ray.origin = hit.position + hit.normal * DELTA;
  // 課題1：鏡面の作成
  vec3 I = ray.direction; // 入射ベクトル
  vec3 N = hit.normal;    // 法線ベクトル
  ray.direction = normalize(I - 2.0 * dot(I, N) * N);
  ray.scatter *= hit.scatter;
}

float fresnel(const in float n, const in float u) {
  float f0 = POW2((n - 1) / (n + 1));
  return f0 + (1 - f0) * POW5(1 - u);
}

// ガラス面
void glass(inout Ray ray, const in Hit hit) {
  ray.depth++;
  float n = 1.5; // 屈折率（ガラスの屈折率を 1.5 と仮定）
  vec3 N;        // 法線ベクトル
  float t = dot(-ray.direction, hit.normal); // 入射ベクトルと法線の内積

  if (t > 0.0f) {
    n = 1.0f / n; // 外部から内部への入射（空気 -> ガラス）
    N = hit.normal;
    t = t;
  } else {
    n = n / 1.0f; // 内部から外部への出射（ガラス -> 空気）
    N = -hit.normal;
    t = -t;
  }

  float F = fresnel(n, t); // フレネル反射率

  if (rand() < F || n * length(cross(N, -ray.direction)) > 1) {
    // フレネル条件または全反射条件を満たす場合は反射
    mirror(ray, hit);
  } else {
    // 屈折
    vec3 I = ray.direction; // 入射ベクトル
    float I_norm = length(I); // 入射ベクトルのノルム ||I||
    float cosThetaI = dot(-I, N) / I_norm; // 入射角のコサイン
    float sin2ThetaT = POW2(n) * (1.0 - POW2(cosThetaI)); // 屈折角のサインの2乗

    if (sin2ThetaT <= 1.0) { // 屈折が可能な場合
      float cosThetaT = sqrt(1.0 - sin2ThetaT); // 屈折角のコサイン
      vec3 T = normalize(-n * I + (n * I_norm * cosThetaI - cosThetaT) * N); // 屈折ベクトル

      ray.origin = hit.position - N * DELTA; // 屈折の始点
      ray.direction = T; // 屈折方向を設定
      ray.scatter *= hit.scatter; // 散乱成分を更新
    } else {
      // 屈折が不可能（全反射）の場合は反射
      mirror(ray, hit);
    }
  }
}

// Image Based Lighting
void background(inout Ray ray, inout Hit hit) {
  ray.depth = DEPTH_MAX;
  hit.emission =
      texture(background_image,
              vec2(-atan(ray.direction.x, ray.direction.z) / (2 * PI),
                   acos(ray.direction.y) / PI))
          .rgb;
}

vec3 random_direction() {
  float theta = rand() * 2.0 * PI;
  float phi = acos(2.0 * rand() - 1.0);

  float x = sin(phi) * cos(theta);
  float y = sin(phi) * sin(theta);
  float z = cos(phi);

  return normalize(vec3(x, y, z));
}

// 完全拡散反射面
void diffuse(inout Ray ray, const in Hit hit) {
  if (dot(-ray.direction, hit.normal) < 0) {
    ray.depth = DEPTH_MAX;
    ray.scatter = vec3(0.0f);
    return;
  }
  ray.depth++;
  float u = rand();
  float z = sqrt(u);
  float d = sqrt(1 - u);
  float phi = rand() * 2.0f * PI;
  vec3 random_vector = random_direction();

  vec3 tangent =
      normalize(random_vector - dot(random_vector, hit.normal) * hit.normal);
  vec3 bitangent = normalize(cross(hit.normal, tangent));

  ray.direction = normalize(tangent * d * cos(phi) + hit.normal * z +
                            bitangent * d * sin(phi));

  ray.origin = hit.position + hit.normal * DELTA;
  ray.scatter *= hit.scatter;
}

void main() {
  xors = texture(seed_image, gl_FragCoord.xy / group_num.xy);

  vec4 color_present =
      (current_sample == 1)
          ? vec4(0.0f)
          : texture(input_image, gl_FragCoord.xy / group_num.xy);

  const vec3 eye = vec3(0.0f, 0.0f, 18.0f);

  const int n_sphere = 2;
  const Sphere spheres[n_sphere] =
      Sphere[n_sphere](Sphere(vec3(0.0f), 4.0f, vec3(0.75f), vec3(0), GLASS),
                       Sphere(vec3(0.0f, -10000.05f, 0.0f), 9996.0f,
                              vec3(0.75f), vec3(0), DIFFUSE));

  mat3 M1 =
      mat3(cos(theta), 0, sin(theta), 0, 1, 0, -sin(theta), 0, cos(theta));

  mat3 M2 = mat3(1, 0, 0, 0, cos(phi), -sin(phi), 0, sin(phi), cos(phi));

  for (int i = 0; i < sample_max; i++) {
    vec4 color_next = vec4(0.0f);

    vec3 position_screen = vec3(
        float(group_idx.x + rand()) / group_num.x * 16.0f - 8.0f,
        float(group_idx.y + rand()) / group_num.y * 9.0f - 4.5f, eye.z - 9.0f);

    Ray ray = Ray(M1 * M2 * (eye + vec3(move_x, move_y, 0)),
                  M1 * M2 * (normalize(position_screen - eye)), vec3(1.0f), 0);

    Hit hit = Hit(1000.0f, vec3(0.0f), vec3(0.0f), vec3(0.0f), vec3(0.0f),
                  BACKGROUND);

    while (ray.depth < DEPTH_MAX) {
      for (int i = 0; i < n_sphere; i++) {
        hitSphere(spheres[i], ray, hit);
      }

      switch (hit.material) {
      case BACKGROUND:
        background(ray, hit);
        break;
      case DIFFUSE:
        diffuse(ray, hit);
        break;
      case MIRROR:
        mirror(ray, hit);
        break;
      case GLASS:
        glass(ray, hit);
        break;
      }

      color_next.rgb += hit.emission * ray.scatter;

      hit.t = 10000.0f;
      hit.material = BACKGROUND;
    }

    // 平均値の逐次計算
    color_present += (color_next - color_present) / (current_sample + i);
  }

  input_color = color_present;
  seed_value = xors;
}


Overwriting assets/glsl/fragment_shader_path_trace.glsl


In [32]:
%%file assets/glsl/fragment_shader_post_process.glsl
#version 330 core

#define POW2(X) ((X) * (X))

out vec4 output_color;

uniform sampler2D input_image;
uniform float luminance_average;
uniform float luminance_max;
uniform float key_value;

ivec2 group_num = ivec2($width, $height);

// Tone Mapping
vec4 toneMap(const in vec4 color) {
  float luminance = 0.27 * color.r + 0.67 * color.g + 0.06 * color.b;
  float luminance_fixed = key_value * (luminance / luminance_average);
  float luminance_max_fixed = key_value * (luminance_max / luminance_average);

  float luminance_mapped = luminance_fixed *
                           (1 + luminance_fixed / POW2(luminance_max_fixed)) /
                           (1 + luminance_fixed);

  vec3 color_fixed = color.rgb * (luminance_mapped / luminance);

  return vec4(clamp(color_fixed, 0, 1), 1.0f);
}

// Gamma Correction
vec4 gammaCorrect(const in vec4 color, const in float gamma) {
  float c = 1.0f / gamma;
  return vec4(pow(color.r, c), pow(color.g, c), pow(color.b, c), 1.0f);
}

void main() {
  vec4 color = texture(input_image, gl_FragCoord.xy / group_num.xy);
  color = toneMap(color);
  output_color = gammaCorrect(color, 2.2);
}


Overwriting assets/glsl/fragment_shader_post_process.glsl


In [21]:
%%file app/server.py
import asyncio
import json

from websockets.server import serve

from render import Context


class WebSocket:
    def __init__(self, context):
        self.context = context

    async def task(self, websocket):
        # キャンセルされるまでサンプリングとレンダリング結果画像の送信を繰り返す
        i = 0
        while True:
            print(["-", "/", "|", "\\"][i % 4], "\r", end="")
            try:
                self.context.current_sample = i * self.context.sample_per_frame + 1
                i += 1
                next_frame = i * self.context.sample_per_frame

                sample_max = self.context.sample_per_frame
                if self.context.max_spp:
                    if next_frame > int(self.context.max_spp):
                        sample_max = (
                            int(self.context.max_spp) % self.context.sample_per_frame
                        )
                        next_frame = int(self.context.max_spp)

                self.context.render(sample_max)

                await asyncio.gather(
                    # レンダリング結果画像を送信する（識別子：0000）
                    websocket.send(b"0000" + self.context.get_binary()),
                    # 現在の1画素あたりのサンプル数を送信する（識別子：0001）
                    websocket.send(b"0001" + bytes(next_frame)),
                )

                if self.context.max_spp:
                    if next_frame >= int(self.context.max_spp):
                        break
            except RuntimeError as e:
                print("Runtime Error:", e)
                break
            except ValueError as e:
                print("ValueError:", e)
                break

    async def echo(self, websocket):
        current_task = None
        print("init")
        print("current_task: ", current_task)

        # クライアントからの接続要求を待ち受ける
        while True:
            message = json.loads(await websocket.recv())
            if "theta" in message:
                self.context.theta = message["theta"]
            if "phi" in message:
                self.context.phi = message["phi"]
            if "moveX" in message:
                self.context.move_x = message["moveX"]
            if "moveY" in message:
                self.context.move_y = message["moveY"]
            if "maxSpp" in message:
                self.context.max_spp = message["maxSpp"]
            if "keyValue" in message:
                self.context.key_value = message["keyValue"]

            for task in asyncio.all_tasks():
                if task.get_coro().__name__ == "task" and not task.done():
                    task.cancel()

            # レンダリングタスクを実行する
            current_task = asyncio.create_task(self.task(websocket))

            print("task assigned")
            print("current_task: ", current_task)

    async def main(self, host, port):
        async with serve(self.echo, host, port):
            print("Listening at: ", f"ws://{host}:{port}")
            await asyncio.Future()  # run forever


if __name__ == "__main__":
    ctx = Context(width=960, height=540, sample_per_frame=64)
    ctx.bind_data(env_map_path="assets/hdr/museum_of_ethnography_1k.hdr")
    ctx.create_program()
    ws = WebSocket(ctx)
    asyncio.run(ws.main("127.0.0.1", 8030))


Overwriting app/server.py


In [22]:
%%file app/render.py
import io
import os
import platform
from string import Template

import cv2
import numpy as np
import moderngl


class Context:
    ATTACHMENT_INDEX_OUTPUT_COLOR = 0
    ATTACHMENT_INDEX_INPUT_COLOR = 1
    ATTACHMENT_INDEX_SEED_VALUE = 2
    TEXTURE_UNIT_INPUT_IMAGE = 1
    TEXTURE_UNIT_SEED_IMAGE = 2
    TEXTURE_UNIT_BACKGROUND_IMAGE = 3

    def __init__(self, width=960, height=540, sample_per_frame=1):
        kwargs = {
            "standalone": True,
            "require": 330,
        }
        if platform.system() == "Linux" and "LD_PRELOAD" not in os.environ:
            kwargs["backend"] = "egl"
        self.context = moderngl.create_context(**kwargs)

        self.width = width
        self.height = height
        self.sample_per_frame = sample_per_frame

        with open("assets/glsl/vertex_shader.glsl", encoding="utf-8") as vs_f:
            self.vertex_shader_str = vs_f.read()
        with open(
            "assets/glsl/fragment_shader_path_trace.glsl", encoding="utf-8"
        ) as fs_f:
            self.fragment_shader_str = fs_f.read()
        with open(
            "assets/glsl/fragment_shader_post_process.glsl", encoding="utf-8"
        ) as fs_f:
            self.post_process_str = fs_f.read()

        self.current_sample = 1
        self.theta = 0
        self.phi = 0
        self.move_x = 0
        self.move_y = 0
        self.max_spp = 0
        self.key_value = 0.18

        self.switch = 0
        self.output_image = None
        self.input_image_list = None
        self.seed_image_list = None

        self.program_path_trace = None
        self.program_post_process = None
        self.vao_path_trace = None
        self.vao_post_process = None
        self.fbo = None

    def bind_data(self, env_map_path):
        data = np.zeros((self.height, self.width, 4)).astype("float32").tobytes()

        # 送信用画像（トーンマップおよびガンマ変換適用済み）
        self.output_image = self.context.texture(
            (self.width, self.height), components=4, dtype="f4"
        )

        # サンプリングを再開するために用いる raw 画像
        self.input_image_list = [
            self.context.texture((self.width, self.height), 4, data, dtype="f4"),
            self.context.texture((self.width, self.height), 4, data, dtype="f4"),
        ]

        seed = (
            np.random.default_rng(0)
            .integers(
                low=0, high=2**32, size=(self.width, self.height, 4), dtype=np.uint32
            )
            .tobytes()
        )

        # 乱数のシード画像（各画素で別々のシード値を使用）
        self.seed_image_list = [
            self.context.texture((self.width, self.height), 4, seed, dtype="u4"),
            self.context.texture((self.width, self.height), 4, seed, dtype="u4"),
        ]

        # 環境マップ画像
        env_map = cv2.imread(env_map_path, cv2.IMREAD_UNCHANGED)
        env_map = cv2.cvtColor(env_map, cv2.COLOR_BGRA2RGBA)
        env_map = env_map.reshape((env_map.shape[1], env_map.shape[0], 4))
        background_image = self.context.texture(
            (env_map.shape[0], env_map.shape[1]), 4, env_map, dtype="f4"
        )
        background_image.write(data=env_map.astype("float32").tobytes())
        self.context.sampler(texture=background_image).use(
            Context.TEXTURE_UNIT_BACKGROUND_IMAGE
        )

    def create_program(self):
        self.program_path_trace = self.context.program(
            vertex_shader=self.vertex_shader_str,
            fragment_shader=Template(self.fragment_shader_str).substitute(
                width=self.width,
                height=self.height,
            ),
            fragment_outputs={
                "output_color": Context.ATTACHMENT_INDEX_OUTPUT_COLOR,
                "input_color": Context.ATTACHMENT_INDEX_INPUT_COLOR,
                "seed_value": Context.ATTACHMENT_INDEX_SEED_VALUE,
            },
        )
        self.program_post_process = self.context.program(
            vertex_shader=self.vertex_shader_str,
            fragment_shader=Template(self.post_process_str).substitute(
                width=self.width,
                height=self.height,
            ),
            fragment_outputs={
                "output_color": Context.ATTACHMENT_INDEX_OUTPUT_COLOR,
            },
        )
        vbo = self.context.buffer(
            np.array(
                [
                    [-1, -1],
                    [-1, 3],
                    [3, -1],
                ],
                dtype="f4",
            )
        )
        self.vao_path_trace = self.context.vertex_array(
            self.program_path_trace,
            [(vbo, "2f /v", "position_vertices")],
        )
        self.vao_post_process = self.context.vertex_array(
            self.program_post_process,
            [(vbo, "2f /v", "position_vertices")],
        )

    def path_trace(self, sample_max, program):
        program["sample_max"].value = sample_max
        program["current_sample"].value = self.current_sample
        program["theta"].value = self.theta
        program["phi"].value = self.phi
        program["move_x"].value = self.move_x
        program["move_y"].value = self.move_y

        program["input_image"].value = Context.TEXTURE_UNIT_INPUT_IMAGE
        program["seed_image"].value = Context.TEXTURE_UNIT_SEED_IMAGE
        program["background_image"].value = Context.TEXTURE_UNIT_BACKGROUND_IMAGE

        self.fbo = self.context.framebuffer(
            [
                self.output_image,
                self.input_image_list[self.switch],
                self.seed_image_list[self.switch],
            ]
        )
        self.fbo.use()
        self.switch = ~self.switch & 1
        self.context.sampler(
            texture=self.input_image_list[self.switch],
            filter=(moderngl.Context.NEAREST, moderngl.Context.NEAREST),
        ).use(Context.ATTACHMENT_INDEX_INPUT_COLOR)
        self.context.sampler(
            texture=self.seed_image_list[self.switch],
            filter=(moderngl.Context.NEAREST, moderngl.Context.NEAREST),
        ).use(Context.ATTACHMENT_INDEX_SEED_VALUE)
        self.context.clear()
        self.vao_path_trace.render(moderngl.Context.TRIANGLES)

    def post_process(self, luminance_average, luminance_max, program):
        program["input_image"].value = Context.TEXTURE_UNIT_INPUT_IMAGE
        program["luminance_average"].value = luminance_average
        program["luminance_max"].value = luminance_max
        program["key_value"].value = self.key_value

        self.fbo = self.context.framebuffer(
            [
                self.output_image,
                self.input_image_list[self.switch],
                self.seed_image_list[self.switch],
            ]
        )
        self.fbo.use()
        self.switch = ~self.switch & 1
        self.context.sampler(
            texture=self.input_image_list[self.switch],
            filter=(moderngl.Context.NEAREST, moderngl.Context.NEAREST),
        ).use(Context.ATTACHMENT_INDEX_INPUT_COLOR)
        self.context.sampler(
            texture=self.seed_image_list[self.switch],
            filter=(moderngl.Context.NEAREST, moderngl.Context.NEAREST),
        ).use(Context.ATTACHMENT_INDEX_SEED_VALUE)
        self.context.clear()
        self.vao_post_process.render(moderngl.Context.TRIANGLES)

    def read_buffer(self, attachment):
        if self.fbo is None:
            raise RuntimeError("frame buffer object has not been assigned")

        return np.frombuffer(
            self.fbo.read(
                components=4,
                dtype="f4",
                attachment=attachment,
            ),
            dtype="f4",
        ).reshape(self.height, self.width, 4)

    def render(self, sample_max):
        if self.program_path_trace is None:
            raise RuntimeError("program_path_trace has not been created")
        if self.program_post_process is None:
            raise RuntimeError("program_post_process has not been created")
        if self.vao_path_trace is None:
            raise RuntimeError("vao_path_trace has not been assigned")
        if self.vao_post_process is None:
            raise RuntimeError("vao_path_process has not been assigned")
        if self.output_image is None:
            raise RuntimeError("output_image has not been assigned")
        if self.input_image_list is None:
            raise RuntimeError("input_image_list has not been assigned")
        if self.seed_image_list is None:
            raise RuntimeError("seed_image_list has not been assigned")

        if self.fbo is not None:
            self.fbo.release()

        self.path_trace(sample_max, self.program_path_trace)

        buffer = self.read_buffer(Context.ATTACHMENT_INDEX_INPUT_COLOR)
        luminance = (
            0.27 * buffer[:, :, 0] + 0.67 * buffer[:, :, 1] + 0.06 * buffer[:, :, 2]
        )
        luminance_average = np.exp(
            np.mean(np.log(np.finfo(np.float32).tiny + luminance))
        )
        luminance_max = buffer.max()

        self.post_process(luminance_average, luminance_max, self.program_post_process)

        self.switch = ~self.switch & 1

    def get_buffer(self):
        buffer = self.read_buffer(Context.ATTACHMENT_INDEX_OUTPUT_COLOR)
        buffer = np.flipud(buffer)
        buffer = cv2.cvtColor(buffer, cv2.COLOR_RGBA2BGRA)
        buffer = (buffer * 255).astype(np.uint8)
        return buffer

    def get_binary(self):
        buffer = self.get_buffer()
        is_success, binary = cv2.imencode(".jpg", buffer)
        with io.BytesIO(binary) as b:
            return b.getvalue()


Overwriting app/render.py


In [23]:

!(mkdir -p assets/hdr && cd assets/hdr && curl -O https://dl.polyhaven.org/file/ph-assets/HDRIs/hdr/1k/museum_of_ethnography_1k.hdr)


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1619k  100 1619k    0     0  6380k      0 --:--:-- --:--:-- --:--:-- 6401k


In [24]:

import json
from pprint import pprint
from urllib import request

ipinfo = json.loads(request.urlopen("https://ipinfo.io").read())
pprint(ipinfo)


{'city': 'Council Bluffs',
 'country': 'US',
 'hostname': '236.7.170.34.bc.googleusercontent.com',
 'ip': '34.170.7.236',
 'loc': '41.2619,-95.8608',
 'org': 'AS396982 Google LLC',
 'postal': '51502',
 'readme': 'https://ipinfo.io/missingauth',
 'region': 'Iowa',
 'timezone': 'America/Chicago'}


In [25]:
%%file colab/tunnel.py
from ping3 import ping
from pyngrok import conf, ngrok


class Tunnel:
    # Ngrok がサポートするリージョンのリスト
    # cf.) https://ngrok.com/docs/ngrok-agent/config/#region
    region_list = ["us", "eu", "ap", "au", "sa", "jp", "in"]

    def __init__(self, token):
        ngrok.set_auth_token(token)
        self.region_priority_list = Tunnel.region_list

    def calc_region_priority(self):
        key_list = []
        for region in Tunnel.region_list:
            public_url = self.get_public_url(region=region)
            key = float("inf")
            key = ping(public_url.replace("https://", "").split(":")[0])
            ngrok.disconnect(public_url)
            ngrok.kill()
            key_list.append(key)

        self.region_priority_list, _ = zip(
            *sorted(zip(Tunnel.region_list, key_list), key=lambda x: x[1])
        )

        print(self.region_priority_list, _)

    def get_public_url(self, port=80, region=None):
        if region == None:
            region = self.region_priority_list[0]
        pyngrok_config = conf.PyngrokConfig(region=region)
        return ngrok.connect(
            port, proto="http", pyngrok_config=pyngrok_config
        ).public_url


Overwriting colab/tunnel.py


In [41]:

from colab.tunnel import Tunnel

# Ngrok の認証トークンを指定する
token = "2sFd8T8lFkEtmTXljnB3pf13s4X_3j2XRiuS61EYUYdjLtr36"

tunnel = Tunnel(token)

# Google Colaboratory のサーバと最も低遅延で通信できるトンネルを見つける
tunnel.calc_region_priority()

# グローバルアクセス可能なURLを取得する
public_url = tunnel.get_public_url(port=8030)
print(public_url.replace("https://", "wss://"))


('jp', 'eu', 'au', 'ap', 'in', 'sa', 'us') (0.020011186599731445, 0.020017147064208984, 0.020037412643432617, 0.020058870315551758, 0.020917654037475586, 0.02129220962524414, 0.022721290588378906)
wss://f263-34-170-7-236.ngrok-free.app


In [43]:

!python app/server.py


/content/app/server.py:4: DeprecationWarning: websockets.server.serve is deprecated
  from websockets.server import serve
Traceback (most recent call last):
  File "/content/app/server.py", line 90, in <module>
    ctx.create_program()
  File "/content/app/render.py", line 103, in create_program
    self.program_path_trace = self.context.program(
                              ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/moderngl/__init__.py", line 1707, in program
    res.mglo, _members, res._subroutines, res._geom, res._glo = self.mglo.program(
                                                                ^^^^^^^^^^^^^^^^^^
_moderngl.Error: GLSL Compiler failed

fragment_shader
0:154(47): error: `cosThetaI' undeclared
0:154(43): error: operands to arithmetic operators must be numeric
0:154(43): error: operands to arithmetic operators must be numeric
0:154(42): error: operands to arithmetic operators must be numeric
0:154(21): error: operands to arithmetic op